# Brokerage/Tradier Development
- Switch to Paper Trading API

In [1]:
# import dependencies
import numpy as np
import scipy as sp
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib as jl
import datetime as dt
import requests

from icecream import ic

import os
from dotenv import load_dotenv

from ta import add_all_ta_features
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.volume import VolumeWeightedAveragePrice
from ta.momentum import StochRSIIndicator

In [2]:
load_dotenv()
TOKEN = os.getenv('TOKEN')

In [3]:
tickers = ['SPY', 'QQQ']
d = {}

for i in range(len(tickers)): 

    response = requests.get('https://api.tradier.com/v1/markets/history',
        params={'symbol': tickers[i], 'interval': 'daily', 'start': '2019-01-01', 'end': '2023-01-01'},
        headers={'Authorization': TOKEN, 'Accept': 'application/json'}
    )
    json_response = response.json()
    #print("Response Status Code:", response.status_code)
    i = i + 1
    print("progress", i)
    for symbol in tickers:
        d[symbol] = pd.DataFrame.from_records(json_response['history']['day'], index='date')

        indicator_macd = MACD(close = d[symbol]['close'], window_slow=26, window_fast=12, window_sign=9, fillna=True)
        d[symbol]['macd'] = indicator_macd.macd()
        d[symbol]['macd_diff'] = indicator_macd.macd_diff()
        d[symbol]['macd_signal'] = indicator_macd.macd_signal()

        indicator_bb = BollingerBands(close = d[symbol]['close'], window=20, window_dev=2, fillna=True)
        d[symbol]['bb_mavg'] = indicator_bb.bollinger_mavg()
        d[symbol]['bb_hband'] = indicator_bb.bollinger_hband()
        d[symbol]['bb_lband'] = indicator_bb.bollinger_lband()
        d[symbol]['bb_hband_ind'] = indicator_bb.bollinger_hband_indicator()
        d[symbol]['bb_lband_ind'] = indicator_bb.bollinger_lband_indicator()

        indicator_vwap = VolumeWeightedAveragePrice(
            high = d[symbol]['high'],
            low = d[symbol]['low'],
            close = d[symbol]['close'],
            volume = d[symbol]['volume'],
            window=14, fillna=True)
        d[symbol]['vwap'] = indicator_vwap.volume_weighted_average_price()

        indicator_stochrsi = StochRSIIndicator(close = d[symbol]['close'], window=14, smooth1=3, smooth2=3, fillna=True)
        d[symbol]['stoch_rsi'] = indicator_stochrsi.stochrsi()
        d[symbol]['stochrsi_d'] = indicator_stochrsi.stochrsi_d()
        d[symbol]['stochrsi_k'] = indicator_stochrsi.stochrsi_k()

progress 1
progress 2


In [4]:
# pull watchlist

response = requests.get('https://api.tradier.com/v1/watchlists/CNBC_INDEX_LIST',
    params={},
    headers={'Authorization': TOKEN, 'Accept': 'application/json'}
)
json_response = response.json()
print('Status Code:', response.status_code)

Status Code: 200


In [5]:
#tickers_df = pd.DataFrame.from_records(json_response['watchlist']['items']['item'])
#tickers = list(pd.read_csv('data/etf_list.csv').Symbol)

In [6]:
response = requests.get('https://api.tradier.com/beta/markets/fundamentals/financials',
    params={'symbols': 'AAPL'},
    headers={'Authorization': TOKEN, 'Accept': 'application/json'}
)
json_response = response.json()
print(response.status_code)
#print(json_response)

200
